# Loading Levada data sets

In [1]:
import pandas as pd
import requests

In [2]:
def get_levada_data(url, indicators):
    
    """
    Loads data from levada website to dataframes
    
    Args: url - a string
          indicators - a list of indicators on a page
    
    Returns: indicators_dict - a dictionary with dataframes,
               where each dataframe contains info on one indicator
    """
    
    url = requests.get(url)
    levada = pd.read_html(url.text)
    
    indicators_dict = {}
    
    for indx in range(len(indicators)):
        
        # dataframe with indicator series
        # this dataframe has columns names: 0, 1, 2, 3
        df = levada[2*indx + 1].T
        
        # rename columns using levada[2*indx]
        df.rename(columns={0: "Month.Year",
                           1: levada[2*indx].iloc[0, 0],
                           2: levada[2*indx].iloc[1, 0],
                           3: levada[2*indx].iloc[2, 0]},
                  inplace=True)
        
        # column Month.Year is float (e.g. 1.2023), 
        # we need to extract Month and Year
        df["Month"] = df["Month.Year"].astype('int')
               
        df["Year"] = (df["Month.Year"] * 10000).astype('int')
        df["Year"] = df["Year"] % 10000
        
        df["Date"] = pd.to_datetime(df['Year'].astype(str) 
                                   +df['Month'].astype(str), 
                                    format='%Y%m')
                
        indicators_dict[indicators[indx]] = df
            
    return indicators_dict       

## Data sources

In [3]:
# Approval of the authorities
authorities_url = "https://www.levada.ru/en/ratings/"

# Indicators on the page
authorities_approval_indicators = [
    'putin',
    'prime_minister',
    'government'
    ]

In [4]:
# Attitudes towards countries
countries_url = "https://www.levada.ru/en/ratings/attitudes-towards-countries/"

# Indicators on the page
countries_attitudes_indicators = [
    'US',
    'EU',
    'Ukraine'
    ]

In [5]:
# Assessment of situation in the country
situation_url = "https://www.levada.ru/en/ratings/assessment-of-situation-in-the-country/"

# Indicators on the page
situation_assessment_indicators = [
    'russia_path'
    ]

## Loading data

### Approval of the authorities

In [6]:
authorities_approval = get_levada_data(authorities_url, authorities_approval_indicators)

In [7]:
authorities_approval['putin']

,Month.Year,Approve,Disapprove,No answer,Month,Year,Date
0,8.1999,31.0,33.0,37.0,8,1999,1999-08-01
1,9.1999,53.0,27.0,20.0,9,1999,1999-09-01
2,10.1999,65.0,20.0,15.0,10,1999,1999-10-01
3,11.1999,80.0,12.0,8.0,11,1999,1999-11-01
4,12.1999,79.0,13.0,8.0,12,1999,1999-12-01
...,...,...,...,...,...,...,...
275,9.2022,77.0,21.0,2.0,9,2022,2022-09-01
276,10.2022,79.0,19.0,2.0,10,2022,2022-10-01
277,11.2022,79.0,18.0,3.0,11,2022,2022-11-01
278,12.2022,81.0,17.0,2.0,12,2022,2022-12-01


In [8]:
# save data to csv
for indicator in authorities_approval_indicators:
    authorities_approval[indicator].to_csv(indicator+".csv")

### Attitudes towards countries

In [9]:
countries_attitudes = get_levada_data(countries_url, countries_attitudes_indicators)

In [10]:
countries_attitudes['US']

,Month.Year,Positive,Negative,Difficult to answer,Month,Year,Date
0,4.1990,75.0,7.0,19.0,4,1990,1990-04-01
1,11.1991,80.0,6.0,14.0,11,1991,1991-11-01
2,8.1992,74.0,8.0,20.0,8,1992,1992-08-01
3,4.1993,72.0,7.0,22.0,4,1993,1993-04-01
4,3.1997,72.0,18.0,10.0,3,1997,1997-03-01
...,...,...,...,...,...,...,...
168,2.2022,31.0,55.0,13.0,2,2022,2022-02-01
169,3.2022,17.0,72.0,11.0,3,2022,2022-03-01
170,5.2022,14.0,75.0,11.0,5,2022,2022-05-01
171,8.2022,19.0,71.0,10.0,8,2022,2022-08-01


In [11]:
# save data to csv
for indicator in countries_attitudes_indicators:
    countries_attitudes[indicator].to_csv(indicator+".csv")

### Assessment of situation in the country

In [12]:
situation_assessment = get_levada_data(situation_url, situation_assessment_indicators)

In [13]:
situation_assessment['russia_path']

,Month.Year,In the right direction,On the wrong track,Difficult to answer,Month,Year,Date
0,7.1996,32.0,45.0,23.0,7,1996,1996-07-01
1,9.1996,21.0,58.0,22.0,9,1996,1996-09-01
2,11.1996,16.0,60.0,24.0,11,1996,1996-11-01
3,1.1997,22.0,63.0,16.0,1,1997,1997-01-01
4,2.1997,14.0,69.0,17.0,2,1997,1997-02-01
...,...,...,...,...,...,...,...
306,9.2022,60.0,27.0,12.0,9,2022,2022-09-01
307,10.2022,64.0,24.0,12.0,10,2022,2022-10-01
308,11.2022,61.0,27.0,12.0,11,2022,2022-11-01
309,12.2022,63.0,24.0,13.0,12,2022,2022-12-01


In [14]:
# save to csv
indicator = 'russia_path'
situation_assessment[indicator].to_csv(indicator+".csv")